In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [6]:
# Load API Credentials
with open('/Users/samspencer/.secret/yelp_api.json') as f:  
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [8]:
LOCATION = 'Lockhart, TX'
TERM = 'BBQ'

In [9]:
JSON_FILE = "Data/TXBBQ.json"
JSON_FILE

'Data/TXBBQ.json'

In [10]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/TXBBQ.json not found. Saving empty list to file.


In [11]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [12]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [13]:
total_results = results['total']
total_results

61

In [14]:
results_per_page = len(results['businesses'])
results_per_page

20

In [15]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

4

In [17]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [18]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,eANZDJ5b801wlfeZTUGzyA,blacks-barbecue-lockhart-lockhart,Black's Barbecue Lockhart,https://s3-media3.fl.yelpcdn.com/bphoto/V6wlGx...,False,https://www.yelp.com/biz/blacks-barbecue-lockh...,1673,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.0,"{'latitude': 29.88609, 'longitude': -97.67289}",[],$$,"{'address1': '215 N Main St', 'address2': '', ...",+15123982712,(512) 398-2712,541.445592
1,Q-aLHWVyO7IdVaxVNWvmxg,terry-blacks-bbq-lockhart-lockhart,Terry Black's BBQ Lockhart,https://s3-media1.fl.yelpcdn.com/bphoto/y14kX2...,False,https://www.yelp.com/biz/terry-blacks-bbq-lock...,44,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 29.893506186396223, 'longitude': ...",[],NaN,"{'address1': '900 N Colorado St', 'address2': ...",+15126684617,(512) 668-4617,1307.431304
2,O4SrowLGiyvrF_oTefQfwA,kreuz-market-lockhart,Kreuz Market,https://s3-media2.fl.yelpcdn.com/bphoto/_tcXEu...,False,https://www.yelp.com/biz/kreuz-market-lockhart...,952,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"{'latitude': 29.889954039326714, 'longitude': ...",[],$$,"{'address1': '619 N Colorado St', 'address2': ...",+15123982361,(512) 398-2361,973.701031
3,71IcJf-dZ54QdSIqC7Q7eQ,smittys-market-lockhart,Smitty's Market,https://s3-media1.fl.yelpcdn.com/bphoto/5a818J...,False,https://www.yelp.com/biz/smittys-market-lockha...,955,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"{'latitude': 29.883638381958, 'longitude': -97...",[],$$,"{'address1': '208 S Commerce St', 'address2': ...",+15123989344,(512) 398-9344,573.112193
4,Bb2YUVvHBDtfGGGHJGKGCA,rileys-pit-bbq-lockhart,Riley's Pit BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/gzMpCr...,False,https://www.yelp.com/biz/rileys-pit-bbq-lockha...,34,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 29.869819192624544, 'longitude': ...",[],NaN,"{'address1': '1326 S Colorado', 'address2': No...",+15123985001,(512) 398-5001,1599.589956


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
56,7zRhI9QZ72z09v_WZAOMEw,hardihoods-eatery-and-drinkery-evo-kyle,Hardihoods Eatery & Drinkery @ EVO,https://s3-media2.fl.yelpcdn.com/bphoto/ndILEv...,False,https://www.yelp.com/biz/hardihoods-eatery-and...,14,"[{'alias': 'newamerican', 'title': 'American (...",2.5,"{'latitude': 30.0301848, 'longitude': -97.8538...",[],NaN,"{'address1': '3200 Kyle Crossing', 'address2':...",+15125239009,(512) 523-9009,23642.959781
57,xM9mCSBSk-0-YaT0tszrpQ,kfc-kyle-2,KFC,https://s3-media2.fl.yelpcdn.com/bphoto/kmJfMK...,False,https://www.yelp.com/biz/kfc-kyle-2?adjust_cre...,20,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"{'latitude': 30.00958085956445, 'longitude': -...","[delivery, pickup]",$,"{'address1': '20925 Interstate Hwy 35', 'addre...",+15122686316,(512) 268-6316,22735.198310
58,SAb1ZXYMY7-HM3l3wgX9jg,wendys-kyle-2,Wendy's,https://s3-media1.fl.yelpcdn.com/bphoto/Qu_WP2...,False,https://www.yelp.com/biz/wendys-kyle-2?adjust_...,16,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,"{'latitude': 30.012156003329803, 'longitude': ...",[delivery],NaN,"{'address1': '20584 Ih 35', 'address2': None, ...",+15125963319,(512) 596-3319,22812.691523
59,XSTpqsFOkWe7nBFCgpiL2g,popeyes-louisiana-kitchen-kyle-2,Popeyes Louisiana Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/XOPV4E...,False,https://www.yelp.com/biz/popeyes-louisiana-kit...,106,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"{'latitude': 30.011973, 'longitude': -97.860128}","[delivery, pickup]",$,"{'address1': '20259 IH 35', 'address2': '', 'a...",+15122681210,(512) 268-1210,22809.960323
60,6zuozyVB73h_5qoklBK8tA,mcdonalds-kyle,McDonald's,https://s3-media1.fl.yelpcdn.com/bphoto/ae53J0...,False,https://www.yelp.com/biz/mcdonalds-kyle?adjust...,65,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"{'latitude': 30.011287, 'longitude': -97.862819}",[delivery],$,"{'address1': '5325 Kyle Center Dr', 'address2'...",+15122681121,(512) 268-1121,22963.756174


In [22]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0